In [1]:
import numpy as np
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
df = pd.read_csv("tips.csv")

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Regresyon için Önişleme

In [4]:
df = pd.concat([df, pd.get_dummies(df["sex"],prefix="sex")], axis=1)
df = pd.concat([df, pd.get_dummies(df["day"],prefix="day")], axis=1)
df = pd.concat([df, pd.get_dummies(df["time"],prefix="time")], axis=1)
df = pd.concat([df, pd.get_dummies(df["smoker"],prefix="smoker")], axis=1)
df.head()  #encoding islemleri gerceklestiriliyor

,total_bill,tip,sex,smoker,day,time,size,sex_Female,sex_Male,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch,smoker_No,smoker_Yes
0,16.99,1.01,Female,No,Sun,Dinner,2,1,0,0,0,1,0,1,0,1,0
1,10.34,1.66,Male,No,Sun,Dinner,3,0,1,0,0,1,0,1,0,1,0
2,21.01,3.50,Male,No,Sun,Dinner,3,0,1,0,0,1,0,1,0,1,0
3,23.68,3.31,Male,No,Sun,Dinner,2,0,1,0,0,1,0,1,0,1,0
4,24.59,3.61,Female,No,Sun,Dinner,4,1,0,0,0,1,0,1,0,1,0


In [5]:
#olceklenecek kolonlari listeliyoruz
columns_to_scale = ['tip', 'size', 'total_bill']

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_columns = pd.DataFrame(scaler.fit_transform(df[columns_to_scale]),columns=columns_to_scale)
scaled_columns.describe()
#olcekleyip ayri bir dataframe haline getirildi
#kategorik veriler dusuruldu

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,tip,size,total_bill
count,244.000000,244.000000,244.000000
mean,0.222031,0.313934,0.350145
std,0.153738,0.190220,0.186477
min,0.000000,0.000000,0.000000
25%,0.111111,0.200000,0.215281
50%,0.211111,0.200000,0.308442
75%,0.284722,0.400000,0.441087
max,1.000000,1.000000,1.000000


In [7]:
df.drop(["total_bill", "tip", "size", "smoker", "sex","day", "time"], axis = 1, inplace = True)
df = pd.concat([df, scaled_columns], axis = 1)
#tum dusurulme encodinglerden sonra versi setinin son hali

In [8]:
df.head()

,sex_Female,sex_Male,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch,smoker_No,smoker_Yes,tip,size,total_bill
0,1,0,0,0,1,0,1,0,1,0,0.001111,0.2,0.291579
1,0,1,0,0,1,0,1,0,1,0,0.073333,0.4,0.152283
2,0,1,0,0,1,0,1,0,1,0,0.277778,0.4,0.375786
3,0,1,0,0,1,0,1,0,1,0,0.256667,0.2,0.431713
4,1,0,0,0,1,0,1,0,1,0,0.290000,0.6,0.450775


In [9]:
df.drop(["sex_Female","time_Dinner", "smoker_No"], axis=1, inplace=True)

In [10]:
Y = df.tip
X = df.loc[:,df.columns!="tip"]

In [11]:
df.head()

,sex_Male,day_Fri,day_Sat,day_Sun,day_Thur,time_Lunch,smoker_Yes,tip,size,total_bill
0,0,0,0,1,0,0,0,0.001111,0.2,0.291579
1,1,0,0,1,0,0,0,0.073333,0.4,0.152283
2,1,0,0,1,0,0,0,0.277778,0.4,0.375786
3,1,0,0,1,0,0,0,0.256667,0.2,0.431713
4,0,0,0,1,0,0,0,0.290000,0.6,0.450775


# **Model Eğitme**

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1)

In [13]:
from sklearn.svm import SVR
svregressor = SVR()
svregressor.fit(X_train, y_train)
predsvr = svregressor.predict(X_test)

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Değerlendirme

In [14]:
from sklearn.metrics import r2_score
print(r2_score(y_test, predsvr))
#r2 score kullaniliyor

0.40038890919630676


In [15]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, predsvr))
#error'a bakiliyor

0.013646030337072866


In [16]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test, predsvr))

0.08969319507521124


# Doğrulama 

In [18]:
from sklearn.model_selection import cross_val_score
cross_val_score(svregressor, X, Y, cv=10).mean()

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: 

0.32994986440401897

# Tuning

Grid Search Cross Validation

In [19]:
import sklearn
from sklearn import metrics
sklearn.metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'accuracy', 'roc_auc', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'brier_score_loss', 'adjusted_rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted'])

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
param_grid = [ {"C":[0.0001, 0.01, 0.1, 1, 10], "kernel":["poly", "linear", "rbf"], "gamma":[0.0001, 0.01, 0.1, 1, 10]}]
grid_search = GridSearchCV(svregressor, param_grid, cv=5)
grid_search.fit(X, Y)
grid_search.best_estimator_

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [21]:
grid_search.best_score_

0.38697873545894024

# # Random Search Cross Validation
Burada 0'la dört arası bir uniform distribution içinde C değerlerini arıyoruz.
uniform[loc, loc+scale]

In [22]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
distributions = dict(C = uniform(loc = 0, scale = 4), kernel=["poly","rbf","linear"])
random = RandomizedSearchCV(svregressor, distributions, random_state=0)
search = random.fit(X, Y)
search.best_params_

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma wil

{'C': 3.3770629943240693, 'kernel': 'rbf'}

In [23]:
df.head()

,sex_Male,day_Fri,day_Sat,day_Sun,day_Thur,time_Lunch,smoker_Yes,tip,size,total_bill
0,0,0,0,1,0,0,0,0.001111,0.2,0.291579
1,1,0,0,1,0,0,0,0.073333,0.4,0.152283
2,1,0,0,1,0,0,0,0.277778,0.4,0.375786
3,1,0,0,1,0,0,0,0.256667,0.2,0.431713
4,0,0,0,1,0,0,0,0.290000,0.6,0.450775


# Sınıflandırma

In [24]:
Y = df.sex_Male
X = df.loc[:,df.columns!="sex_Male"]

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, stratify = Y)
#stratify ile setlerin esit sekilde dagilmasini sagliyoruz

In [27]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
predsvc = svc.predict(X_test)

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predsvc))

              precision    recall  f1-score   support

           0       0.40      0.22      0.29         9
           1       0.65      0.81      0.72        16

   micro avg       0.60      0.60      0.60        25
   macro avg       0.53      0.52      0.50        25
weighted avg       0.56      0.60      0.57        25



In [30]:
from sklearn.model_selection import cross_val_score
cross_val_score(svc, X, Y, cv=10).mean()

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: 

0.5666086956521739

# Stratified K Fold

In [31]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(svc, X, Y, cv=skf)
print("skorlar:\n{}".format(scores))
print("skorların ortalaması:\n{}".format(scores.mean()))

skorlar:
[0.64       0.64       0.64       0.36       0.36       0.56
 0.64       0.65217391 0.52173913 0.65217391]
skorların ortalaması:
0.5666086956521739


C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: 

In [32]:
from sklearn.model_selection import GridSearchCV
param_grid = [ {"C":[0.0001, 0.01, 0.1, 1, 10], "kernel":["poly", "linear", "rbf"], "gamma":[0.0001, 0.01, 0.1, 1, 10]}]
grid_search_c = GridSearchCV(svc, param_grid, cv = 5, scoring = "accuracy")
grid_search_c.fit(X, Y)
grid_search_c.best_estimator_

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [33]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
distributions = dict(C = uniform(loc = 0, scale = 4), kernel=["poly","rbf","linear"])
random_c = RandomizedSearchCV(svc, distributions, random_state=0)
search_c = random_c.fit(X, Y)
search_c.best_params_

C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Nano\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma wil

{'C': 0.22685190926977272, 'kernel': 'poly'}

In [35]:
search_c.best_score_

0.6434426229508197